In [24]:
HF_USR_NAME = 'shirwu'
TOOL_QA_ROOT = ''

### Upload to Huggingface

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

level = 'hard'
dataset = 'scirex'

dataset_dir = f'{dataset}-{level}.jsonl'
hf_dataset_name = f'toolqa_{dataset}_{level}'

df = pd.read_json(dataset_dir, lines=True)
df.head()

df['answer'] = df['answer'].apply(lambda x: str(x))
dataset = Dataset.from_pandas(df)

In [3]:
dataset_dict = DatasetDict({'train': dataset})
# push to hf for the ease for using dspy
# dataset_dict.push_to_hub(repo_id=hf_dataset_name, private=True)

## Setting Up

* ToolQA

Before loading our datasets and going to the execution part, we'll need to configure the `lm` in `dspy.settings`. For the purpose of this notebook we'll be using `gpt-4o`.

In [4]:
import os
import dspy
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning) 


dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
        temperature=0
    )
)

## Defining Signature

In [5]:
class ToolQASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question with a short response. 
    """
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )


## Loading Datasets

In [6]:
from random import sample
from dspy.datasets import DataLoader

dl = DataLoader()

In [7]:
tool_qa = dl.from_huggingface(
    f'{HF_USR_NAME}/' + hf_dataset_name,
    split="train",
    input_keys=("question", "answer"),
)

In [8]:
len(tool_qa)

100

In [9]:
import random
# set seed
random.seed(42)

train_idx = random.sample(range(len(tool_qa)), 40)
remaining_idx = list(set(range(len(tool_qa))) - set(train_idx))
test_idx = random.sample(remaining_idx, 60)

toolqa_train = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in train_idx]
]
toolqa_test = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in test_idx]
]

## Setting Up Tools

We'll setup `Avatar` modules for both signatures and all the `tools` can be used by each of the dataset. `Tool` is a pydantic model that Avatar expects the `tools` to be composed as more specifically it have 4 fields:

* `name` : Name of the tool
* `input_type` : Type of input the tool accepts
* `output_type` : Type of output the tool returns
* `tool` : The actual tool object

In [10]:
import os
import time
import uuid
import numpy as np
import jsonlines
from concurrent.futures import ProcessPoolExecutor
import sentence_transformers
import chromadb
from os import path as osp
from chromadb.config import Settings

EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/scirex-v2")
CHROMA_COLLECTION_NAME = "all"
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_HTTP_PORT = "8000"
FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/scirex/Preprocessed_Scirex.jsonl")

def sentence_embedding(model, texts):
    embeddings = model.encode(texts)
    return embeddings

def create_chroma_db(chroma_server_host, chroma_server_http_port, collection_name):
    chroma_client = chromadb.Client(Settings(
        chroma_api_impl="rest",
        chroma_server_host=chroma_server_host,
        chroma_server_http_port=chroma_server_http_port,
    ))
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def create_chroma_db_local(persist_directory, collection_name):
    chroma_client = chromadb.PersistentClient(path=persist_directory)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def insert_to_db(texts, model_name, cuda_idx, db):
    # use cpu
    model = sentence_transformers.SentenceTransformer(model_name, device='cpu')
    # model = sentence_transformers.SentenceTransformer(model_name, device=f"cuda:{cuda_idx}")

    batch_embeddings = []
    batch_texts = []
    start_time = time.time()
    print(f"Total Articles to process: {len(texts)}, Current Thread: {cuda_idx}.")
    for i, text in enumerate(texts):
        # 2. generate embedding
        embeddings = sentence_embedding(model, text).tolist()

        batch_embeddings.append(embeddings)
        batch_texts.append(text)
        # 3. add to vectorstore per 500 articles or last article
        if i % 100 == 0 or i == len(texts)-1:
            batch_ids = [str(uuid.uuid1()) for _ in batch_texts]
            db.add(
                embeddings=batch_embeddings,
                documents=batch_texts,
                ids = batch_ids
            )
            batch_embeddings = []
            batch_texts = []
            print(f"Completed Processing article count: {i}, Current Thread: {cuda_idx}, Time took: {time.time() - start_time}.")
    print(f"Thread {cuda_idx} Completed. Total time took for thread: {time.time() - start_time}.")


# Multi-processing
def query_llm(query, is_local=True, start=None, end=None):
    cuda_idxes = [0]
    number_of_processes = len(cuda_idxes)
    input_texts = []
    db = create_chroma_db_local(CHROMA_PERSIST_DIRECTORY, CHROMA_COLLECTION_NAME)
    with open(FILE_PATH, 'r') as f:
        for item in jsonlines.Reader(f):
            input_texts.append(item["content"])
    # input_texts = np.array_split(input_texts, number_of_processes)

    args = ((input_texts[i], EMBED_MODEL_NAME, cuda_idxes[i], is_local) for i in range(number_of_processes))

    # if there is no file under the directory "/localscratch/yzhuang43/ra-llm/retrieval_benchmark/data/chroma_db/agenda", insert the data into the db
    # You should run insert_to_db the first time!
    if len(os.listdir(CHROMA_PERSIST_DIRECTORY)) == 0:
        insert_to_db(input_texts, model_name=EMBED_MODEL_NAME, cuda_idx=0, db=db)

    input_paths = np.array_split(input_texts, number_of_processes)
    with ProcessPoolExecutor(number_of_processes) as executor:
        executor.map(insert_to_db, args)
    # use cpu
    model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device='cpu')
    # model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device=f"cuda:0")
    query_embedding = sentence_embedding(model, query).tolist()
    results = db.query(query_embeddings=query_embedding, n_results=3)
    retrieval_content = [result for result in results['documents'][0]]
    # print(retrieval_content)
    retrieval_content = '\n'.join(retrieval_content)
    return retrieval_content

query = "What is an atom"
print(query_llm(query))

paragraph : Sentence Level For representing a document , one can split it up into sentences , with each memory slot encoding one sentence . Both the key and the value encode the entire sentence as a bag - of - words . As the key and value are the same in this case , this is identical to a standard MemNN and this approach has been used in several papers .
paragraph : Window Level Documents are split up into windows of words ; in our tasks we only include windows where the center word is an entity . Windows are represented using bag - of - words . Window representations for MemNNs have been shown to work well previously . However , in Key - Value MemNNs we encode the key as the entire window , and the value as only the center word , which is not possible in the MemNN architecture . This makes sense because the entire window is more likely to be pertinent as a match for the question ( as the key ) , whereas the entity at the center is more pertinent as a match for the answer ( as the valu

In [11]:
from dspy.predict.avatar import Tool, Avatar
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool

def RETRIEVE(query: str) -> str:
    """If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE(\'Which method achieves the highest PCK score?\') returns relevant paper paragraph and meta data."""
    return query_llm(query)

tools = [
    Tool(
        tool=StructuredTool.from_function(RETRIEVE),
        name="RETRIEVE",
        desc="If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE('Which method achieves the highest PCK score?') returns relevant paper paragraph and meta data."
    ),
    Tool(
        tool=GoogleSerperAPIWrapper(),
        name="WEB_SEARCH",
        desc="If you have a question, you can use this tool to search the web for the answer."
    ),
    Tool(
        tool=ArxivAPIWrapper(),
        name="ARXIV_SEARCH",
        desc="Pass the arxiv paper id to get the paper information.",
        input_type="Arxiv Paper ID",
    )
]

Once we have defined our `tools`, we can now create an `Avatar` object by passing the `tools` and `signature`. It takes 2 more optional parameters `verbose` and `max_iters`. `verbose` is used to display the logs and `max_iters` is used to control the number of iterations in multi step execution. 

An avatar agent stops the tool usage iteration once it reaches `max_iters` or when it prompts `Finish`. You can also create custom tools too, all you need to make sure is:

* You pass is a class object.
* Implements `__init__` and `run` method.
* Must take 1 string a input and returns 1 string as output.

If your tool doesn't return or takes input a string then you can make a custom wrapper to take care of that for now. In future we'll try to enable a diverse tool usage.

In [12]:
actor_agent = Avatar(
    tools=tools,
    signature=ToolQASignature,
    verbose=False,
    max_iters=10
)

In [13]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy
import tqdm
import logging
import warnings
import os

# Set up logging
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Disable all INFO logging
logging.getLogger().setLevel(logging.WARNING)

# Silence all loggers that might be chatty
loggers_to_silence = [
    "httpx",
    "httpcore",
    "openai",
    "arxiv",
    "dspy",
    "langchain",
    "langchain_community",
    "requests",
    "urllib3",
    "tiktoken",
    "asyncio",
    "faiss",
    "anthropic"
]

for logger_name in loggers_to_silence:
    logging.getLogger(logger_name).setLevel(logging.WARNING)

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Disable tokenizer parallelism warning

## Evaluation

Open enden QA tasks are hard to evaluate on rigid metrics like exact match. So, we'll be using an improvised LLM as Judge for the evaluation of our model on test set.

In [14]:
class Evaluator(dspy.Signature):
    """Please act as an impartial judge to evaluate whether the answer is correct based on the ground truth answer"""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
    )
    reference_answer: str = dspy.InputField(
        prefix="Ground Truth Answer:",
        desc="Ground truth answer to the question.",
    )
    answer: str = dspy.InputField(
        prefix="Answer:",
        desc="Answer to the question given by the model.",
    )
    rationale: str = dspy.OutputField(
        prefix="Rationale:",
        desc="Explanation of why the answer is correct or incorrect.",
    )
    is_correct: float = dspy.OutputField(
        prefix="Correct:",
        desc="Whether the answer is correct. Give 0 if incorrect, 1 if correct, (0, 1) if partially correct.",
    )


evaluator = dspy.TypedPredictor(Evaluator)


def metric(example, prediction, trace=None):  
    # We found sometimes the ground truth answers are incomplete or the answer
    # is part of the ground truth answer. Therefore, for better comparison, 
    # we use a continuous value for the correct score   
    acc = float(
        evaluator(
            question=example.question,
            answer=prediction.answer,
            reference_answer=example.answer
        ).is_correct
    ) 
    print(prediction.answer, '|', example.answer, '=>', acc)
    return acc

print(toolqa_train[0])
metric(toolqa_train[0], prediction=dspy.Example(answer='physics'))

Example({'question': 'Which method achieves the highest PCK score on Leeds_Sports_Poses dataset for Pose_Estimation task?', 'answer': 'Pyramid_Residual_Modules__PRMs_'}) (input_keys={'question', 'paper_id'})
physics | Pyramid_Residual_Modules__PRMs_ => 0.0


0.0

For evaluation we can't use `dspy.Evaluate`, reason being that `Avatar` changes it's signature per iteration by adding the actions and it's results to it as fields. So we can create our own hacky thread safe evaluator for it.

In [15]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class APICallMetrics:
    timestamp: datetime
    tool_name: str
    tokens_in: int = 0
    tokens_out: int = 0
    execution_time: float = 0.0

@dataclass
class AvatarMetrics:
    total_calls: int = 0
    total_tokens_in: int = 0
    total_tokens_out: int = 0
    total_execution_time: float = 0.0
    calls_by_tool: Dict[str, int] = field(default_factory=dict)
    api_call_history: List[APICallMetrics] = field(default_factory=list)
    
    def add_call(self, metrics: APICallMetrics):
        self.total_calls += 1
        self.total_tokens_in += metrics.tokens_in
        self.total_tokens_out += metrics.tokens_out
        self.total_execution_time += metrics.execution_time
        self.calls_by_tool[metrics.tool_name] = self.calls_by_tool.get(metrics.tool_name, 0) + 1
        self.api_call_history.append(metrics)
    
    def merge(self, other: 'AvatarMetrics'):
        """Merge another AvatarMetrics instance into this one"""
        self.total_calls += other.total_calls
        self.total_tokens_in += other.total_tokens_in
        self.total_tokens_out += other.total_tokens_out
        self.total_execution_time += other.total_execution_time
        for tool, count in other.calls_by_tool.items():
            self.calls_by_tool[tool] = self.calls_by_tool.get(tool, 0) + count
        self.api_call_history.extend(other.api_call_history)

    def estimate_cost(self, model_name: str = "gpt-4") -> float:
        pricing = {
            "gpt-4": {"input": 2.5, "output": 10.0},
        }
        if model_name not in pricing:
            raise ValueError(f"Unknown model: {model_name}")
        
        rates = pricing[model_name]
        input_cost = (self.total_tokens_in / 1000000) * rates["input"]
        output_cost = (self.total_tokens_out / 1000000) * rates["output"]
        return input_cost + output_cost

class AvatarWithMetrics(Avatar):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metrics = AvatarMetrics()
        self.tokenizer = tiktoken.encoding_for_model("gpt-4")
    
    def _count_tokens(self, text: str) -> int:
        try:
            return len(self.tokenizer.encode(str(text)))
        except Exception as e:
            logger.warning(f"Error counting tokens: {e}")
            return 0

    def _wrapped_tool_call(self, tool, input_text: str) -> str:
        start_time = time.time()
        tokens_in = self._count_tokens(input_text)
        
        try:
            result = tool.run(input_text)
        except Exception as e:
            logger.error(f"Tool execution error: {e}")
            raise
        finally:
            execution_time = time.time() - start_time
            tokens_out = self._count_tokens(str(result))
            
            metrics = APICallMetrics(
                timestamp=datetime.now(),
                tool_name=tool.name,
                tokens_in=tokens_in,
                tokens_out=tokens_out,
                execution_time=execution_time
            )
            self.metrics.add_call(metrics)
            
        return result

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = super().__call__(*args, **kwargs)
        total_time = time.time() - start_time
        
        metrics = APICallMetrics(
            timestamp=datetime.now(),
            tool_name="main_llm",
            tokens_in=self._count_tokens(str(args) + str(kwargs)),
            tokens_out=self._count_tokens(str(result)),
            execution_time=total_time
        )
        self.metrics.add_call(metrics)
        
        return result

def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for example in test_set:
            def process_with_metrics(example=example):
                try:
                    avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
                    prediction = avatar(**example.inputs().toDict())
                    return metric(example, prediction), avatar.metrics
                except Exception as e:
                    print(e)
                    return 0, AvatarMetrics()

            futures.append(executor.submit(process_with_metrics))

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            score, metrics = future.result()
            total_score += score
            # Only combine token counts and call counts, not execution times
            combined_metrics.total_calls += metrics.total_calls
            combined_metrics.total_tokens_in += metrics.total_tokens_in
            combined_metrics.total_tokens_out += metrics.total_tokens_out
            for tool, count in metrics.calls_by_tool.items():
                combined_metrics.calls_by_tool[tool] = combined_metrics.calls_by_tool.get(tool, 0) + count
            combined_metrics.api_call_history.extend(metrics.api_call_history)
    
    total_execution_time = time.time() - start_time
    combined_metrics.total_execution_time = total_execution_time

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        try:
            avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
            prediction = avatar(**example.inputs().toDict())
            score = metric(example, prediction)
            total_score += score
            # Combine metrics from this run
            for call in avatar.metrics.api_call_history:
                combined_metrics.add_call(call)
        except Exception as e:
            print(e)

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def format_metrics_report(metrics: AvatarMetrics, model_name: str = "gpt-4") -> str:
    cost = metrics.estimate_cost(model_name)
    
    report = f"""
Avatar Execution Metrics Report
==============================
Execution Time: {metrics.total_execution_time:.2f} seconds
Total API Calls: {metrics.total_calls}
Total Tokens: {metrics.total_tokens_in + metrics.total_tokens_out:,} ({metrics.total_tokens_in:,} in, {metrics.total_tokens_out:,} out)
Estimated Cost: ${cost:.4f}

Average Time per Call: {metrics.total_execution_time / metrics.total_calls:.2f} seconds

Tool Usage Breakdown:
-------------------
"""
    for tool, count in sorted(metrics.calls_by_tool.items()):
        report += f"{tool}: {count} calls\n"

    return report

## One-shot result

In [16]:
score, metrics = multi_thread_executor(toolqa_test, ToolQASignature)

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method that achieves the highest F1 score on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is ACE + document-context with an F1 score of 94.6. | CVT___Multi-Task => 0.0
EffNet-L2 (SAM) achieves the highest Percentage_correct score of 96.08 on the CIFAR-100 dataset for the Image Classification task. | Res2NeXt-29 => 0.0
The Paragraph_vector method for the Question Answering task has been evaluated on datasets such as SQuAD, SelQA, WikiQA, NewWikiQA, and InforBoxQA. | WikiQA => 0.5
The method EASE achieves the highest Recall@50 score of 0.428 on the Million Song Dataset for the Collaborative Filtering task. | Mult-VAE_PR => 0.0
The ACF-WIDER method achieves the highest AP score for the Face Detection task on the WiderFace dataset. | WIDER_Face__Easy_ => 0.0
The highest reported Mean_IoU score on the CamVid dataset for Semantic Segmentation is 66.1%. | PSPNet => 0.0
The Discriminative Unsupervised Feature Learning with Convolutional Neural Networks method is evalua

Processing examples:   2%|▏         | 1/60 [01:42<1:40:50, 102.55s/it]

The TuckER method is evaluated on four standard link prediction datasets: FB15k, FB15k-237, WN18, and WN18RR. | FB15k-237 => 0.5
MuZero achieves the highest Score score on the Atari_2600_Name_This_Game dataset for the Atari_Games task. | IQN => 0.0
The VAT_EntMin method for Semi-Supervised Image Classification is evaluated on the MNIST, SVHN, and CIFAR-10 datasets. | CIFAR-10__4000_Labels => 0.0
The SVDCNN method for text classification is evaluated on several large-scale datasets, including AG's News Corpus, Sogou News Corpus, DBPedia Ontology Dataset, Yelp Reviews, Yahoo! Answers, and Amazon Reviews. | AG_News => 0.5
The ConvNet method for Keypoint Detection on the Pascal3D dataset is evaluated using the area under the PCK-over-alpha curve as a function of the number of training annotations. | Mean_PCK => 0.0
The available tools did not provide the specific dataset on which the DDQN__tuned__noop method achieves the highest Score score for the Atari_Games task. Further specific resear

Processing examples:   3%|▎         | 2/60 [01:48<44:16, 45.80s/it]   

The PNN method evaluation metrics on the Bing_News dataset for the Click-Through Rate Prediction task are not explicitly mentioned in the retrieved documents. However, common evaluation metrics for CTR prediction tasks typically include accuracy, precision, recall, and F1-score. | AUC, Log_Loss => 0.0


Processing examples: 100%|██████████| 60/60 [01:51<00:00,  1.86s/it]

The CyCADA method is evaluated on the SYNTHIA_Fall-to-Winter dataset for the Image-to-Image Translation task using metrics such as mean Intersection over Union (mIoU), frequency weighted Intersection over Union (fwIoU), and pixel accuracy. These metrics assess the performance of the model in terms of semantic segmentation and pixel-level adaptation. | Per-pixel_Accuracy, fwIOU, mIoU => 1.0


In [17]:
# print(f"Average Score on ArxivQA before opitmization: {aqa_score:.2f}")
print(f"Test Score: {score:.2f}")
print(format_metrics_report(metrics))

Test Score: 0.22

Avatar Execution Metrics Report
Execution Time: 112.91 seconds
Total API Calls: 60
Total Tokens: 85,847 (1,702 in, 84,145 out)
Estimated Cost: $0.8457

Average Time per Call: 1.88 seconds

Tool Usage Breakdown:
-------------------
main_llm: 60 calls



## Optimization

For the optimization of the `Actor` we'll be using `AvatarOptimizer`. It's a DSPy implementation of the [Avatar](https://github.com/zou-group/avatar/) method that optimizes the `Actor` for the given `tools` using a comparator module that optimizes Actor instruction. Note, that Actor is the Module that directs tool execution and flow, it's not the signature that we are passing. It doesn't optimize the instruction of the signature we pass. It takes the following parameters:

* `metric`: Metric that we'll be optimizing for
* `max_iters`: Maximum number of iterations for the optimizer
* `lower_bound`: Lower bound for the metric to classify example as negative
* `upper_bound`: Upper bound for the metric to classify example as positive
* `max_positive_inputs`: Maximum number of positive inputs sampled for comparator
* `max_negative_inputs`: Maximum number of negative inputs sampled for comparator
* `optimize_for`: Whether we want to maximize the metric or minimize it during optimization

Once the optimizer is done we can get the optimized actor and use it for the evaluation.

In [18]:
from batched_optimizer import AvatarOptimizerWithMetrics

iterative_monkey = AvatarOptimizerWithMetrics(
    metric=metric,
    max_iters=2,
    max_negative_inputs=10,
    max_positive_inputs=10,
    lower_bound=0.5,
    upper_bound=0.5
)

In [ ]:
result = iterative_monkey.compile(
    student=actor_agent,
    trainset=toolqa_train,
    batch_size=4,
)

Processing batch 1 of 4...


Processing examples:   0%|          | 0/40 [00:00<?, ?it/s]

OmniPose achieves the highest PCK score of 99.5% on the Leeds Sports Poses dataset for the Pose Estimation task. | Pyramid_Residual_Modules__PRMs_ => 0.0
The method that achieves the highest SSIM score on the Vid4 - 4x upscaling dataset for Video Super-Resolution is EvTexture+ with an SSIM score of 0.8983. | VESPCN => 0.0
The method that achieves the highest MRR score on the FB15k dataset for the Link Prediction task is AutoKGE with an MRR of 0.861. | TuckER => 0.0
The IQN method is evaluated on 57 Atari 2600 games in the ALE (Atari Learning Environment). | Atari_2600_Kung-Fu_Master => 0.5
The X-Transformer achieved the highest BLEU score of 46.63 on the WMT2014 English-German dataset for the Machine Translation task. | Weighted_Transformer__large_ => 0.0
The ByteNet method is evaluated on the English-to-German WMT translation task for Machine Translation. | WMT2014_English-French => 0.0
The Subgraph_embeddings method is evaluated on the WebQuestions dataset using the average F1 score 

Processing examples:   5%|▌         | 2/40 [00:57<18:05, 28.58s/it]

The Duel_noop method is evaluated on 57 Atari games for the Atari_Games task. | Atari_2600_Time_Pilot => 0.0
The DeepFM method achieves the highest Log_Loss score for the Click-Through Rate Prediction task on the Criteo dataset. The Criteo dataset is a well-known ad tech industry benchmarking dataset used for evaluating CTR prediction models. | Criteo => 1.0
The Duel_noop method is evaluated on 57 Atari games for the Atari_Games task. | Atari_2600_Ms__Pacman => 0.0
The PSENet-1s method is evaluated on the SCUT-CTW1500 dataset using precision, recall, and F-measure metrics. | F-Measure => 0.5
The Mult-DAE method is evaluated on the Netflix dataset using the NDCG@100 metric for the Collaborative Filtering task. | Recall_20, Recall_50 => 0.0
The Duel_hs method evaluation datasets for Atari Games could not be found in the available resources. It seems that specific information about the datasets used for evaluating the Duel_hs method on Atari Games is not readily available in the searched 

...

Processing examples:  12%|█▎        | 5/40 [00:27<02:50,  4.87s/it]

The A3C-CTS method is evaluated on the Atari 2600 suite, which includes games like Montezuma's Revenge and other hard exploration games with sparse rewards. | Atari_2600_Venture => 0.5
The method that achieves the highest Medium_Human-Normalized_Score score on the Atari-57 dataset for the Atari_Games task is GDI-H3 with a score of 9620.33%. | Ape-X => 0.0
The CornerNet-Squeeze method is evaluated on the PASCAL VOC and MS COCO datasets for the Real-Time Object Detection task. | COCO => 0.5
The Duel_noop method is evaluated on 57 Atari games, including both human and noop start settings. | Atari_2600_Ms__Pacman => 0.0
The highest AP_0_5 score on the PASCAL-Person-Part dataset for the Multi-Human Parsing task is 59.70%, achieved by the NAN method. | NAN => 0.0
PSENet-1s is evaluated on the SCUT-CTW1500 dataset using metrics such as F-measure, precision, and recall. The F-measure achieved by PSENet is noted to be 82.2%. | F-Measure => 0.5
The DANN method evaluation metrics for the Multi-Do

Processing examples: 100%|██████████| 40/40 [00:45<00:00,  1.14s/it]

The datasets on which the PFF method is evaluated for Image Super-Resolution are not explicitly mentioned in the available resources. Common datasets for evaluating super-resolution methods include Set5, Set14, and DIV2K, but specific information about PFF is missing. | Set14_-_4x_upscaling => 0.5
Average Score: 0.3125


Generated new instruction: New Instruction: You will be given `Tools`, which is a list of resources available to accomplish the `Goal`. Your task is to carefully select the most appropriate tool for each user query and determine the specific input values to provide. When deciding on an `Action`, ensure it includes the chosen tool and the input query tailored to the task. Remember, you can choose not to use any tools and provide the final answer directly if it is more efficient. Additionally, you may use a tool multiple times with different input queries if needed.

To enhance your performance, focus on precise tool selection and query formulation. Begin by conducting a preliminary analysis of the query to determine its specificity and relevance to the available tools. For example, use `ARXIV_SEARCH` when detailed academic information or specific paper IDs are required, and opt for `WEB_SEARCH` for broader queries. Ensure that your queries are specific and directly related to the task, 

In [20]:
optimized_actor_agent = result["agent"]
optimization_metrics = result["metrics"]

# Now you can process the metrics
print(f"Total optimization cost: ${optimization_metrics['total_cost']:.4f}")
print(f"Final score achieved: {optimization_metrics['final_score']:.3f}")

# Analyze per-iteration performance
for iteration in optimization_metrics['iteration_details']:
    print(f"\nIteration {iteration['iteration']}:")
    print(f"Score: {iteration['score']:.3f}")
    print(f"Comparator tokens in: {iteration['comparator_metrics']['tokens_in']}")
    print(f"Comparator tokens out: {iteration['comparator_metrics']['tokens_out']}")
    print(f"Feedback tokens in: {iteration['feedback_metrics']['tokens_in']}")
    print(f"Feedback tokens out: {iteration['feedback_metrics']['tokens_out']}")
    print(f"Execution time: {iteration['execution_time']:.2f}s")

Total optimization cost: $1.8880
Final score achieved: 0.312

Iteration 0:
Score: 0.312
Comparator tokens in: 24814
Comparator tokens out: 409
Feedback tokens in: 546
Feedback tokens out: 289
Execution time: 320.18s

Iteration 1:
Score: 0.312
Comparator tokens in: 34178
Comparator tokens out: 344
Feedback tokens in: 669
Feedback tokens out: 321
Execution time: 158.41s


Now we can evaluate our actor module, for this we've provided an implementation of thread safe evaluator that we above as part of class method of `AvatarOptimizer`.

In [21]:
# iterative_monkey.thread_safe_evaluator(toolqa_test, optimized_actor_agent)
batch_num = 4
iterative_monkey.thread_safe_evaluator_batch(toolqa_test, optimized_actor_agent,batch_num)

Processing batch 1 of 4...


Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method EASE achieves the highest Recall@50 score of 0.428 on the Million Song Dataset for the Collaborative Filtering task. | Mult-VAE_PR => 0.0
The highest F1 score achieved on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is 85.11%. | CVT___Multi-Task => 0.0
The method achieving the highest Number_of_params score on the WikiText-2 dataset for the Language Modelling task is the adversarial training mechanism proposed by Dilin Wang, Chengyue Gong, and Qiang Liu, which achieved a test perplexity score of 38.07. | AWD-LSTM-DOC => 0.0
The TuckER method is evaluated on standard link prediction datasets, including FB15K-237 and WN18RR. | FB15k-237 => 0.5
The method that achieves the highest Mean IoU score on the CamVid dataset for the Semantic Segmentation task is SERNet-Former with a score of 84.62. | PSPNet => 0.0
The 300D_NTI-SLSTM-LSTM_encoders method for Natural Language Inference is evaluated on the Stanford Natural Language Inference (SNLI) dataset, MultiGenre

Processing examples:   2%|▏         | 1/60 [00:30<30:06, 30.63s/it]

The current state-of-the-art method achieving the highest score on the Atari 2600 Name This Game dataset for the Atari Games task is MuZero, with a score of 157177.85. | IQN => 0.0
The specific evaluation metrics for the Paragraph_vector__lexical_overlap___dist_output_ method on the QASent dataset for the Question Answering task could not be found in the available resources. It may require access to specific academic papers or datasets that detail these metrics. | MAP, MRR => 0.0
Unable to find specific evaluation metrics for the Prior_Duel_hs method on the Atari_2600_Alien dataset for the Atari_Games task. Consider checking academic papers or specific research articles related to this method for detailed information. | Score => 0.0
The Inception_V2 method is typically evaluated on the ImageNet dataset using metrics such as Top-1 Accuracy, Top-5 Accuracy, Precision, Recall, and F1 Score for the Image Classification task. | Top_1_Accuracy, Top_5_Accuracy => 0.5
The MemNNs__ensemble_ met

...

Processing examples:  32%|███▏      | 19/60 [00:37<00:43,  1.06s/it]

('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'action_6': "ValueError('json output should start and end with { and }')"})
The DRCN method is typically evaluated using metrics such as Peak Signal-to-Noise Ratio (PSNR) and Structural Similarity Index (SSIM) on the Set5 dataset for 4x upscaling in the Image Super-Resolution task. | MOS, PSNR, SSIM => 0.67
The current state-of-the-art on the SNLI dataset for Natural Language Inference is achieved by Neural Tree Indexers for Text Understanding. However, specific details about the highest Parameters score are not readily available from the search results. | 300D_Residual_stacked_encoders => 0.0


Processing examples: 100%|██████████| 60/60 [00:39<00:00,  1.50it/s]

The DQN_noop method is evaluated on the Atari 2600 games, which are part of the Arcade Learning Environment (ALE). This includes a wide range of games used for benchmarking reinforcement learning algorithms. | Atari_2600_River_Raid => 0.0


0.22233333333333333

In [22]:
iterative_monkey.thread_safe_evaluator(toolqa_test, optimized_actor_agent)

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method that achieves the highest Mean IoU score on the CamVid dataset for the Semantic Segmentation task is SERNet-Former with a score of 84.62. | PSPNet => 0.0
The Snips method for Speech Recognition is evaluated on the Hey-Snips dataset and internal datasets. | LibriSpeech_test-clean => 0.0
The highest Train Accuracy score on the SNLI dataset for the Natural Language Inference task is achieved by the Neural Tree Indexers for Text Understanding and EFL (Entailment as Few-shot Learner) models, both with a Test Accuracy of 93.1%. | __Unigram_and_bigram_features => 0.0
The IQN method achieves the highest Score score on the 57 Atari 2600 games dataset in the ALE (Arcade Learning Environment). | Atari_2600_Atlantis => 0.0
The Stacked Hourglass Networks method achieves the highest PCK_0_2 score for the Pose Estimation task on the FLIC dataset. | FLIC_Elbows => 0.0
The method achieving the highest Number_of_params score on the WikiText-2 dataset for the Language Modelling task is the adv

Processing examples:   2%|▏         | 1/60 [00:12<11:55, 12.13s/it]

MuZero achieves the highest score on the Atari_2600_Name_This_Game dataset for the Atari_Games task. | IQN => 0.0
The iBOWIMG_baseline method's highest Percentage_correct score for the Visual_Question_Answering task could not be determined from the available resources. | COCO_Visual_Question_Answering__VQA__real_images_1_0_multiple_choice => 0.0
The novel directed hypergraph neural network method achieves the highest accuracy on the Cora dataset for the node classification task. | GCN => 0.0
The VAT_EntMin method for Semi-Supervised Image Classification does not have specific datasets mentioned in the available search results. Further detailed search or access to specific papers might be required to find this information. | CIFAR-10__4000_Labels => 0.0
The shallow-and-wide network model achieves the highest error score on the Yelp Binary classification dataset for the Sentiment Analysis task, with a performance of 95.9% as reported in the paper "Do Convolutional Networks need to be Dee

Processing examples:   7%|▋         | 4/60 [00:30<06:47,  7.27s/it]

The VGG_Resnet_LACE_BiLSTM acoustic model trained on SWB, Fisher, and CH datasets, along with the N-gram and RNNLM language model trained on Switchboard, Fisher, Gigaword, and Broadcast, is evaluated on datasets such as LibriSpeech for the Speech Recognition task. | swb_hub_500_WER_fullSWBCH => 0.0
The evaluation metrics for the Prior_Duel_hs method on the Atari_2600_Alien dataset for the Atari_Games task could not be found in the available resources. It may require specific access to research papers or datasets that were not retrieved in the search. | Score => 0.0
The S-Norm method for the Question Answering task is evaluated on datasets such as SQuAD, SelQA, WikiQA, NewWikiQA, and InforBoxQA. | TriviaQA => 0.0
The DRCN method is typically evaluated on metrics such as Peak Signal-to-Noise Ratio (PSNR) and Structural Similarity Index (SSIM) for the Set5 dataset in the 4x upscaling Image Super-Resolution task. However, specific details for DRCN on Set5 were not found in the search resul

Processing examples: 100%|██████████| 60/60 [00:41<00:00,  1.44it/s]

The available searches did not provide specific information on the dataset where SVDCNN achieves the highest error score for Sentiment Analysis. Further detailed searches or specific academic papers might be needed to find this information. | Yelp_Fine-grained_classification => 0.0


0.16666666666666666